<a href="https://colab.research.google.com/github/Gapilesh/BlinkKey_FinalProject/blob/main/Eyeblink8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Mount Google Drive

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2: Unzip the Dataset

In [2]:
import zipfile

# Path to the zip file in your drive
zip_path = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8.zip'

# Unzip the dataset into the Eyeblink folder in your drive
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Dataset/Eyeblink')

print("Dataset unzipped successfully!")

Dataset unzipped successfully!


Step 3: List Folders in the Dataset

In [3]:
import os

# Path to the unzipped dataset
dataset_path = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8'

# List folders (e.g., '1', '2', '3', '4', '8', '9', '10', '11')
folders = os.listdir(dataset_path)
print("Folders in the dataset:", folders)

# Check contents of one folder (for example, folder '1')
sample_folder = os.path.join(dataset_path, '1')
sample_files = os.listdir(sample_folder)
print("Files in folder '1':", sample_files)

Folders in the dataset: ['1', '10', '11', '2', '3', '4', '8', '9']
Files in folder '1': ['26122013_223310_cam.avi', '26122013_223310_cam.tag', '26122013_223310_cam.txt']


Step 4: Extract Frames from a Video (Using a Lower Frame Limit and a Drive Output Folder)

In [7]:
!pip install opencv-python

import cv2
import os

def extract_frames(video_path, output_folder, frame_limit=300):
    # Create output folder if it doesn't exist—in your Drive to persist files
    os.makedirs(output_folder, exist_ok=True)

    # Open the video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if (not ret) or (frame_count >= frame_limit):
            break

        # Save each frame as an image
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {video_path}")

# Example: Extract frames from folder '1', adjust the output folder path as needed
video_path = os.path.join(dataset_path, '1', '26122013_223310_cam.avi')
output_folder = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink8/eyeblink_frames/1'
extract_frames(video_path, output_folder, frame_limit=300)

Extracted 300 frames from /content/drive/MyDrive/Dataset/Eyeblink/eyeblink8/1/26122013_223310_cam.avi


Step 5: Parse Annotations

In [21]:
def parse_annotations(annotation_path):
    blink_labels = []
    with open(annotation_path, 'r') as file:
        for line in file.readlines():
            # Use comma (or adjust to appropriate delimiter, e.g., whitespace)
            parts = line.strip().split(' ') # Used space to split
            if len(parts) >= 2:
                # You can modify this to compute a label based on the decimal value if needed.
                frame_number = parts[0]
                # For demonstration, we label a blink if the timestamp value exceeds a threshold (example logic)
                timestamp = float(parts[1])
                label = 1 if timestamp > 0.3 else 0
                blink_labels.append(label)
    return blink_labels

# Example: Parse annotations for folder '1'
annotation_path = os.path.join(dataset_path, '1', '26122013_223310_cam.txt')
blink_labels = parse_annotations(annotation_path)
print("Blink labels for folder '1' (first 10):", blink_labels[:10])

Blink labels for folder '1' (first 10): [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


Step 6: Define a Lightweight Model

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, TimeDistributed, Flatten

# Define a lightweight model with input shape matching resized frame size (176, 176)
model = Sequential([
    TimeDistributed(Conv2D(16, (3, 3), activation='relu'), input_shape=(None, 176, 176, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(32, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(32, return_sequences=True),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, None, 174, 174, 16)  │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, None, 87, 87, 16)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, None, 85, 85, 32)    │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, None, 42, 42, 32)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, None, 56448)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 32)            │       7,229,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 1)             │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,234,689 (27.60 MB)

 Trainable params: 7,234,689 (27.60 MB)

 Non-trainable params: 0 (0.00 B)

Step 7: Load Frames & Train in Segments

In [25]:
import numpy as np

def load_frames_and_labels(frames_folder, annotation_file, frame_limit=100):
    frames = []
    labels = parse_annotations(annotation_file)  # Use the single annotation file

    for i in range(frame_limit):
        frame_path = os.path.join(frames_folder, f"frame_{i:04d}.jpg")
        frame = cv2.imread(frame_path)
        if frame is None:
            break
        # Resize frame to the dimension expected by the model
        frame = cv2.resize(frame, (176, 176))
        frame = frame / 255.0  # Normalize to [0,1]
        frames.append(frame)

    # Use only as many labels as there are loaded frames
    return np.array(frames), np.array(labels[:len(frames)])

# Load a segment (first 100 frames) from folder '1'
frames_folder = '/content/drive/MyDrive/Dataset/Eyeblink/eyeblink_frames/1'
annotation_file = os.path.join(dataset_path, '1', '26122013_223310_cam.txt')
frames, labels = load_frames_and_labels(frames_folder, annotation_file, frame_limit=100)
print("Frames shape:", frames.shape)
print("Labels shape:", labels.shape)

# Add a batch dimension (here one sequence; for a more robust training, consider using a generator)
frames_seq = frames[np.newaxis, ...]  # Shape: (1, sequence_length, 176, 176, 3)
labels_seq = labels[np.newaxis, ...]  # And corresponding labels

# Train the model on this small segment to test resource usage
model.fit(frames_seq, labels_seq, epochs=5)

Frames shape: (100, 176, 176, 3)
Labels shape: (100,)
Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/sequential_1/time_distributed_1_2/strided_slice_99/StridedSliceGrad defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambd

Step 8: Save and Evaluate the Model

In [ ]:
# Save the trained model to your Google Drive
model.save('/content/drive/MyDrive/eyeblink_model.h5')
print("Model saved successfully!")

# Evaluate the model on the same segment
loss, accuracy = model.evaluate(frames_seq, labels_seq)
print("Loss:", loss)
print("Accuracy:", accuracy)